---
# **SETUP**
---


In [1]:
%%capture
from sentence_transformers import SentenceTransformer
from pca import pca
import pickle
import numpy as np
import scipy as sp
import pandas as pd
import trimesh
from trimesh import creation
import math
import time

In [2]:
%%capture
# Change to override local model loading
load_local = True

# Change if you don't want to have a downloaded version of the model
# For reference sentence-transformers/all-mpnet-base-v2 is 438.5 MB
download_model = True

try:
  if load_local:
    model = pickle.load(open("emb-model.pkl", "rb"))
  else:
    raise Exception
except:
  model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
  if (download_model):
    pickle.dump(model, open("emb-model.pkl", "wb"))

---
# **FUNCTIONS**
---


In [3]:
def embed(model, texts, anchors):
    if len(anchors) != 4:
        print("Incorrect number of anchors")
        return

    embeddings = model.encode(texts + anchors)
    print(np.linalg.norm(embeddings[0]))
    data_frame = pd.DataFrame(embeddings)
    pca_model = pca(n_components=4)
    dict = pca_model.fit_transform(data_frame)
    embeddings = np.array(dict["PC"])
    anchors = np.array(embeddings[-4:])
    return embeddings, anchors

In [4]:
def project(embeddings, anchors):
    A, B, C, D = anchors

    # Compute the vectors that define the plane defined by the anchors
    AB = B - A
    AC = C - A
    AD = D - A

    # Get the basis unit vectors for the plane defined by AB and AC
    u = AB + (-AB.dot(AC) / (AB - AC).dot(AC)) * (AB - AC)
    u = u / np.linalg.norm(u)
    v = AC / np.linalg.norm(AC)

    # Find x, y that minimize distance of
    # line segment between D and E (E = xu + yv)
    y = (AD - AD.dot(u) * u).dot(v) / (1 + (v.dot(u) * u).dot(v))
    x = (AD - y * v).dot(u)

    # Get the perspective vector from D to ABC∆
    E = AD - x * u - y * v
    E = E / np.linalg.norm(E)

    # Get the rotations to align E with the W axis
    Ex, Ey, Ez, Ew = E

    # Rotate in the XY and ZW planes first, setting Ex = 0 and Ez = 0
    # T stands for theta
    Θxy = (math.pi / 2 - math.asin(Ey / np.linalg.norm(np.array([Ex, Ey])))) * np.sign(
        Ex
    )
    Θzw = (math.pi / 2 - math.asin(Ew / np.linalg.norm(np.array([Ez, Ew])))) * np.sign(
        Ez
    )

    # Apply the rotations in the XY and ZW planes
    Rxyzw = np.array(
        [
            [math.cos(Θxy), -math.sin(Θxy), 0, 0],
            [math.sin(Θxy), math.cos(Θxy), 0, 0],
            [0, 0, math.cos(Θzw), -math.sin(Θzw)],
            [0, 0, math.sin(Θzw), math.cos(Θzw)],
        ]
    )
    E = E.dot(Rxyzw)
    Ex, Ey, Ez, Ew = E

    # Rotate in the YW plane next, setting Ey = 0
    Θyw = (math.pi / 2 - math.asin(Ew / np.linalg.norm(np.array([Ey, Ew])))) * np.sign(
        Ey
    )
    Ryw = np.array(
        [
            [1, 0, 0, 0],
            [0, math.cos(Θyw), 0, -math.sin(Θyw)],
            [0, 0, 1, 0],
            [0, math.sin(Θyw), 0, math.cos(Θyw)],
        ]
    )
    E = E.dot(Ryw)

    # Translate so that A is the origin
    translated = embeddings - A

    # Rotate so that E is aligned with the W unit vector
    rotated = []
    for emb in translated:
        emb = emb.dot(Rxyzw)
        emb = emb.dot(Ryw)
        rotated.append(emb)

    # Undo translation
    Ar = A.dot(Rxyzw)
    Ar = Ar.dot(Ryw)

    projected = []
    for emb in rotated:
        emb = emb + Ar
        emb = emb[:3]
        projected.append(emb)

    return np.array(projected)

In [22]:
def create_stl(points, cube_size, point_size, buffer, axis_size, resolution):
    point_core_multiplier = 5
    axis_core_multiplier = 3

    cube = creation.box(extents=(2.0, 2.0, 2.0))

    # Scale the points down to accomodate buffer
    scale = 1 - (buffer + point_size) / cube_size
    points = points * scale

    # Points
    for point in points[:-4]:
        sphere = creation.icosphere(subdivisions=3, radius=point_size / cube_size)
        sphere.apply_translation(point)

        cube = cube.difference(sphere)

    # Anchors
    for point in points[-4:]:
        icos = creation.icosahedron()
        icos.apply_scale(point_size / cube_size)
        icos.apply_translation(point)

        cube = cube.difference(icos)

    # Cores and pillars
    for point in points[:-4]:
        core = creation.icosphere(
            subdivisions=3,
            radius=(point_size - resolution * point_core_multiplier) / cube_size,
        )
        core.apply_translation(point)
        pillar = creation.cylinder(
            resolution * point_core_multiplier / cube_size, point_size / cube_size * 2
        )
        pillar.apply_translation(point)

        cube = cube.union(core).union(pillar)

    for point in points[-4:]:
        core = creation.icosahedron()
        core.apply_scale((point_size - resolution * point_core_multiplier) / cube_size)
        core.apply_translation(point)
        pillar = creation.cylinder(
            resolution * point_core_multiplier / cube_size, point_size / cube_size * 2
        )
        pillar.apply_translation(point)

        cube = cube.union(core).union(pillar)

    # Axes
    axis_radius = axis_size / cube_size
    axis_length = 2 * (1 - buffer / cube_size - axis_radius)
    axis_core_radius = (axis_size - resolution * axis_core_multiplier) / cube_size
    Xaxis = creation.cylinder(axis_radius, axis_length)
    Xaxis.apply_transform(
        np.array([[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])
    )
    Yaxis = creation.cylinder(axis_radius, axis_length)
    Yaxis.apply_transform(
        np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
    )
    Zaxis = creation.cylinder(axis_radius, axis_length)

    cube = cube.difference(Xaxis).difference(Yaxis).difference(Zaxis)

    # Add spheres at the end to round the axes
    endpoint = axis_length / 2
    endcaps = [
        [0, 0, endpoint],
        [0, 0, -endpoint],
        [0, endpoint, 0],
        [0, -endpoint, 0],
        [endpoint, 0, 0],
        [-endpoint, 0, 0],
    ]
    for point in endcaps:
        sphere = creation.icosphere(subdivisions=3, radius=axis_radius)
        sphere.apply_translation(point)
        cube = cube.difference(sphere)

        core = creation.icosphere(subdivisions=3, radius=axis_core_radius)
        core.apply_translation(point)

        cube = cube.union(core)

    Xaxis_core = creation.cylinder(axis_core_radius, axis_length)
    Xaxis_core.apply_transform(
        np.array([[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])
    )
    Yaxis_core = creation.cylinder(axis_core_radius, axis_length)
    Yaxis_core.apply_transform(
        np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
    )
    Zaxis_core = creation.cylinder(axis_core_radius, axis_length)

    Xaxis_pillar = creation.box(
        extents=(
            axis_length,
            resolution * axis_core_multiplier / cube_size,
            axis_radius * 2,
        )
    )
    Yaxis_pillar = creation.box(
        extents=(
            resolution * axis_core_multiplier / cube_size,
            axis_length,
            axis_radius * 2,
        )
    )
    Zaxis_pillar = creation.cylinder(
        resolution * axis_core_multiplier / cube_size, axis_length + axis_radius * 2
    )

    cube = (
        cube.union(Xaxis_core)
        .union(Yaxis_core)
        .union(Zaxis_core)
        .union(Xaxis_pillar)
        .union(Yaxis_pillar)
        .union(Zaxis_pillar)
    )

    return cube

---
# **PROCESSING**
---


In [6]:
texts = [
    "kill",
    "love",
    "shrug",
    "eat",
    "wonder",
    "hunt",
    "hold",
    "break",
    "fill",
    "kiss",
    "lie",
    "demand",
    "starve",
    "dance",
    "gaze",
    "whisper",
    "punch",
    "trust",
    "cook",
    "regale",
    "run",
    "hurt",
    "attack",
    "invent",
    "bleed",
    "sing",
    "flirt",
    "dream",
    "die",
    "shoot",
    "steal",
    "instruct",
    "command",
    "lead",
    "imagine",
    "create",
    "shout",
    "build",
    "play",
    "win",
    "lose",
    "draw",
    "drum",
    "wail",
    "accept",
    "reject",
    "tend",
    "hug",
    "garden",
    "grow",
    "write",
    "paint",
    "swim",
    "dress",
    "design",
    "sew",
    "teach",
    "care",
    "leave",
    "bear",
    "joke",
    "sip",
    "gulp",
    "stand",
    "sit",
    "leap",
    "jump",
    "block",
]

# The first 3 anchors end up as points along unit dimensions and the 4th anchor
# defines the perspective from which the projection is calculated
anchors = ["men", "women", "human", "artificial intelligence"]

In [7]:
# Test for duplicates
all = texts + anchors
for item in all:
    instances = [i for i in all if i == item]
    if len(instances) > 1:
        print(item)

In [8]:
%%capture
embeddings, anchors = embed(model, texts, anchors)

In [9]:
projected = project(embeddings, anchors)

In [10]:
# OPTIONAL
# Scale points to fill cube, useful for smaller cubes

if True:
    max_coord = np.max(projected)
    projected = projected / max_coord

In [23]:
# Input desired dimensions in mm

# Length of each side, max = 120mm
cube_size = 50
# Radius, min = 0.025mm
point_size = 2
# Axis size, min = 0.025mm
axis_size = 1
# Buffer, recommended = 7mm (~0.25in)
buffer = 7
# Printer resolution, used to calculate the gap between voids and cores
resolution = 0.048

cube = create_stl(projected, cube_size, point_size, buffer, axis_size, resolution)

In [24]:
%%capture
timestamp = time.strftime("%Y%m%d_%H%M%S")
cube.export("outputs/resin_point_cloud_" + str(cube_size) + "mm_" + timestamp + ".stl")